**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Feb 26 09:26:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 AI_Challenge_Program.rar
 bq-results-20220927-023430-1664246148735
 Code_Sharing
'Colab Notebooks'
'Copy of Cosmetics Inc..gsheet'
'Copy of QUERY.gsheet'
'Cosmetics Inc..gsheet'
 CustomerSurveyData.gsheet
 CustomerSurvey.gsheet
'Dataset for Project:  CONCAT function.gsheet'
'Data Spreadsheet for "Cleaning with Spreadsheets"  (1).gsheet'
'Data Spreadsheet for "Cleaning with Spreadsheets" .gsheet'
'DeliveryTimes DistanceData (1).gsheet'
'DeliveryTimes DistanceData.gsheet'
'Dynamic Dataset.gsheet'
 File_Sharing
 Inventory.csv
'June 2014 Invoices.gsheet'
'KIB - Best in Show (public).gsheet'
'Kitty Survey Feedback (1).gsheet'
'Kitty Survey Feedback (2).gsheet'
'Kitty Survey Feedback.gsheet'
'Making your own visualization - example dataset.gsheet'
'Movie Data Starter Project (1).gsheet'
'Movie Data Starter Project (2).gsheet'
'Movie Data Starter Project (3).gsheet'
'Movie Data Starter Project.gsheet'
 My_Code
 Others
 Photo_20201002
'Population, Latin and Caribbea

**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.17 MiB | 7.45 MiB/s, done.
Resolving deltas: 100% (10403/10403), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-02-26 09:29:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  10.8MB/s    in 15s     

2023-02-26 09:29:51 (10.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/1400.png       
  inflating: data/obj/1400.txt       
  inflating: data/obj/1425.png       
  inflating: data/obj/1425.txt       
  inflating: data/obj/1450.png       
  inflating: data/obj/1450.txt       
  inflating: data/obj/1475.png       
  inflating: data/obj/1475.txt       
  inflating: data/obj/1500.png       
  inflating: data/obj/1500.txt       
  inflating: data/obj/1525.png       
  inflating: data/obj/1525.txt       
  inflating: data/obj/1550.png       
  inflating: data/obj/1550.txt       
  inflating: data/obj/1575.png       
  inflating: data/obj/1575.txt       
  inflating: data/obj/1600.png       
  inflating: data/obj/1600.txt       
  inflating: data/obj/1625.png       
  inflating: data/obj/1625.txt       
  inflating: data/obj/1650.png       
  inflating: data/obj/1650.txt       
  inflating: data/obj/1675.png       
  inflating: data/obj/1675.txt       
  inflating: data/obj/1700.png       
  inflating: 

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.426563 0.373611 0.468750 0.425000

['0', '0.426563', '0.373611', '0.468750', '0.425000']
0 data/obj/1075.txt
0 0.426563 0.373611 0.468750 0.425000
0 0.582031 0.577778 0.273438 0.316667

['0', '0.582031', '0.577778', '0.273438', '0.316667']
1 data/obj/1975.txt
0 0.582031 0.577778 0.273438 0.316667
0 0.350000 0.434722 0.543750 0.475000

['0', '0.350000', '0.434722', '0.543750', '0.475000']
2 data/obj/1025.txt
0 0.350000 0.434722 0.543750 0.475000
0 0.528906 0.511111 0.195312 0.516667

['0', '0.528906', '0.511111', '0.195312', '0.516667']
3 data/obj/1300.txt
0 0.528906 0.511111 0.195312 0.516667
0 0.850000 0.643056 0.178125 0.241667

['0', '0.850000', '0.643056', '0.178125', '0.241667']
4 data/obj/175.txt
0 0.850000 0.643056 0.178125 0.241667
0 0.193750 0.883333 0.175000 0.227778

['0', '0.193750', '0.883333', '0.175000', '0.227778']
4 data/obj/175.txt
0 0.193750 0.883333 0.175000 0.227778
0 0.282031 0.715278 0.029687 0.313889

['0', '0.282031', '0.715278', '0.029687', '0.313889']
4 d

In [13]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/2000.png', 'data/obj/300.png', 'data/obj/1775.png', 'data/obj/925.png', 'data/obj/1900.png', 'data/obj/1650.png', 'data/obj/1825.png', 'data/obj/150.png', 'data/obj/800.png', 'data/obj/1300.png', 'data/obj/825.png', 'data/obj/1450.png', 'data/obj/525.png', 'data/obj/675.png', 'data/obj/1100.png', 'data/obj/1625.png', 'data/obj/425.png', 'data/obj/75.png', 'data/obj/650.png', 'data/obj/50.png', 'data/obj/575.png', 'data/obj/1725.png', 'data/obj/1375.png', 'data/obj/850.png', 'data/obj/1125.png', 'data/obj/250.png', 'data/obj/400.png', 'data/obj/375.png', 'data/obj/975.png', 'data/obj/1150.png', 'data/obj/1250.png', 'data/obj/1925.png', 'data/obj/325.png', 'data/obj/500.png', 'data/obj/450.png', 'data/obj/200.png', 'data/obj/1550.png', 'data/obj/775.png', 'data/obj/1800.png', 'data/obj/700.png', 'data/obj/1000.png', 'data/obj/0.png', 'data/obj/125.png', 'data/obj/475.png', 'data/obj/2050.png', 'data/obj/1500.png', 'data/obj/1175.png', 'data/obj/2025.png', 'data/obj/1400.png', 

In [14]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.850767), count: 7, class_loss = 0.178646, iou_loss = 0.099450, total_loss = 0.278095 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.925658), count: 2, class_loss = 0.183629, iou_loss = 0.008810, total_loss = 0.192439 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 299419, rewritten_bbox = 0.000668 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.839834), count: 4, class_loss = 0.004601, iou_loss = 0.083164, total_loss = 0.087765 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000003, iou_loss = 0.000000, total_loss = 0.000003 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.0